In [ ]:
"""
Algorithm 1 LEO-Based EC Algorithm
1: Begin
2: // Initialization
3: g←0 ; // the generation index
4: Initialize population Xg and evaluate the fitness;
5: Initialize the weights of ANN randomly;
6: Initialize arch as an empty set; // to store solution pairs
7: While stop criteria not satisfied Do
8:		g←g+ 1;
9:		// Individual Evolution
10:		Sample r uniformly on [0,1];
11:		If g> 1 and r< lp Then
12:			newX ← Evolve Xg by learning-aided evolutionary operator;
13:		Else
14:			// use operators in traditional EC, e.g., PSO or DE
15:			newX ← Evolve Xg by traditional evolutionary operator;
16:		End If
17:		Evaluate the fitness of individuals in newX;
18:		// Selection
19:		Xg+1← selection among Xg and newX;
20:		// SEP Collection
21:		For each individual i in Xg+1 Do
22:			If Xg+1,i is better than Xg,i Then
23:				Add (Xg,i , Xg+1,i ) in arch;
24:			End If
25:		End For
26:		If number of SEPs in arch > arch_size Then
27:			arch ← the newest arch_size solution pairs;
28:		End If
29:		// Learning System Update
30:		Train the ANN with all data in arch for one epoch;
31:	End While
32:End
"""

In [24]:
from pymoo.algorithms.moo.moead import MOEAD
from pymoo.problems import get_problem
from pymoo.core.population import Population
from pymoo.optimize import minimize
from keras.layers import Dense
from keras.models import Sequential
from pymoo.core.evaluator import Evaluator
from pymoo.operators.mutation.pm import PM
from pymoo.operators.crossover.sbx import SBX
from pymoo.indicators.igd import IGD
from pymoo.util.ref_dirs import get_reference_directions
import numpy as np
import random

In [25]:
def normalize(data):
    min_val = -100
    max_val = 100
    normalized_data = (data - min_val) / (max_val - min_val)
    return normalized_data


def denormalize(data):
    min_val = -100
    max_val = 100
    denormalized_data = (data * (max_val - min_val)) + min_val
    return denormalized_data

In [26]:
def generate_model(n_var):
    model = Sequential()
    model.add(Dense((3*n_var), input_dim=n_var, activation='sigmoid'))
    model.add(Dense(n_var, activation='sigmoid'))

    model.compile(loss='mse',
                  optimizer='adam', metrics=['accuracy'])
    return model

In [27]:
def dominates(p, q):
    return np.all(p <= q) and np.any(p < q)

In [28]:

def traditional_moead(problem, ref_dirs, pop_size=100, generation=100, verbose=True):

    algorithm = MOEAD(
        ref_dirs,
        crossover=SBX(eta=15, prob=0.9),
        mutation=PM(eta=20, prob=0.5),
        n_neighbors=15,
        prob_neighbor_mating=0.7,
    )
    algorithm.pop_size = pop_size
    res = minimize(problem, algorithm, ("n_gen", generation), verbose=verbose)
    return res.X, res.F

In [29]:
n_obj = 5
n_var = 100
l_bound = -100
h_bound = 100

problem = get_problem('dtlz2', n_var=n_var, n_obj=n_obj)
problem.xl = np.array([-100 for _ in range(100)])
problem.xu = np.array([100 for _ in range(100)])
ref_dirs = get_reference_directions("uniform", n_obj, n_partitions=12)

pop_traditional, pareto_traditional = traditional_moead(
    problem, ref_dirs, pop_size=100, generation=100, verbose=True)

n_gen  |  n_eval  | n_nds  |      eps      |   indicator  
     1 |     1820 |    357 |             - |             -
     2 |     3640 |   1155 |  0.0927391861 |         ideal
     3 |     5460 |   1101 |  0.0819734847 |         ideal
     4 |     7280 |   1048 |  0.1193316019 |         ideal
     5 |     9100 |   1063 |  0.1721565063 |         ideal
     6 |    10920 |    941 |  0.1536462992 |         ideal
     7 |    12740 |   1033 |  0.1210718879 |         ideal
     8 |    14560 |   1164 |  0.2170734466 |         ideal
     9 |    16380 |   1237 |  0.1382983328 |         ideal
    10 |    18200 |   1141 |  0.0929154604 |         ideal
    11 |    20020 |   1189 |  0.0069557838 |         ideal
    12 |    21840 |   1293 |  0.0844080139 |         ideal
    13 |    23660 |   1398 |  0.0259478411 |         ideal
    14 |    25480 |   1355 |  0.1581810941 |         ideal
    15 |    27300 |   1387 |  0.0976211680 |         ideal
    16 |    29120 |   1417 |  0.0970091444 |         ide

In [30]:

def leo_based_moead(problem, ref_dirs, pop_size=100, generation=100, lp=0.1, n_var=100, l_bound=-100, h_bound=100, verbose=True):

    pop = np.random.uniform(l_bound, h_bound, (pop_size, problem.n_var))

    fit = problem.evaluate(pop)

    model = generate_model(n_var)

    archive_x = []
    archive_y = []

    for g in range(generation):
        if g > 0 and random.random() < lp and len(archive_x) > 0:
            pred = model.predict(normalize(pop), verbose=verbose)
            pop_new = denormalize(pred)
            fit_new = problem.evaluate(pop_new)
        else:
            algorithm = MOEAD(
                ref_dirs,
                crossover=SBX(eta=15, prob=0.9),
                mutation=PM(eta=20, prob=0.9),
                sampling=pop,
                n_neighbors=20,
                prob_neighbor_mating=0.7,
            )
            res = minimize(problem, algorithm, ("n_gen", 1), verbose=False)
            pop_new = res.X
            fit_new = res.F
        for i in range(fit_new.shape[0]):
            if dominates(fit_new[i], fit[i]):
                archive_x.append(pop[i])
                archive_y.append(pop_new[i])
        pop = pop_new
        fit = fit_new
        if len(archive_x) > 0:
            archive_x = archive_x[-pop_size:]
            archive_y = archive_y[-pop_size:]
            model.fit(normalize(np.array(archive_x)), normalize(np.array(
                archive_y)), epochs=1, verbose=verbose)
        print(f"gen:  {g}, archive size: {len(archive_x)}", end='       \r')
    return pop, fit

In [31]:
n_obj = 5
n_var = 100
l_bound = -100
h_bound = 100

problem = get_problem('dtlz2', n_var=n_var, n_obj=n_obj)
problem.xl = np.array([-100 for _ in range(100)])
problem.xu = np.array([100 for _ in range(100)])

ref_dirs = get_reference_directions("uniform", n_obj, n_partitions=5)

pop_leo, pareto_leo = leo_based_moead(
    problem, ref_dirs, pop_size=100, generation=100,
    lp=0.1, n_var=n_var, l_bound=l_bound, h_bound=h_bound, verbose=False)

In [32]:
pf_true = problem.pareto_front(ref_dirs)

igd = IGD(pf_true)

print("traditional: ", igd(pareto_traditional))
print("leo-based: ", igd(pareto_leo))

traditional:  691338.0434740282
leo-based:  1240929.576531278
